In [7]:
import numpy as np
import  pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error

In [2]:
def predict_menstrual_cycle(periods):
    average_period = np.mean(periods)
    std_deviation = np.std(periods)
    covariance_matrix = np.cov(periods[:-1], periods[1:])
    predicted_cycle = np.random.multivariate_normal([average_period, average_period], covariance_matrix, 1)[0]
    return predicted_cycle

In [3]:
dataset = pd.read_csv("CycleData.csv")
# print(dataset.head())
length_of_cycle = dataset["LengthofCycle"].tolist()
print(predict_menstrual_cycle(length_of_cycle))

[30.06200183 32.34490373]


In [4]:
ClientId = "2323"
CycleNumber = 1
LengthOfCycle = 28
estimated_day_of_ovulation = 14
Age=22
Weight=56
Height=90
BMI=21.25472411
# Create a dictionary with the user data
user_data = {
    "ClientID": [ClientId],
    "CycleNumber": [CycleNumber],
    "LengthOfCycle": [LengthOfCycle],
    "ClientWeight": [Weight],
    "ClientAge":[Age],
    "ClientBMI":[BMI],
    "ClientHeight":[Height],

}

# Load the existing dataset from a CSV file
existing_data = pd.read_csv("CycleData.csv")

# Create a new DataFrame with the user data
new_data = pd.DataFrame(user_data)

# Concatenate the new data with the existing dataset
combined_data = pd.concat([existing_data, new_data], ignore_index=True)

# Save the combined data to a new CSV file
combined_data.to_csv("Combined_dataset.csv", index=False)




In [5]:
print(combined_data.columns)

Index(['ClientID', 'CycleNumber', 'Group', 'CycleWithPeakorNot',
       'ReproductiveCategory', 'LengthofCycle', 'MeanCycleLength',
       'EstimatedDayofOvulation', 'LengthofLutealPhase', 'FirstDayofHigh',
       'TotalNumberofHighDays', 'TotalHighPostPeak', 'TotalNumberofPeakDays',
       'TotalDaysofFertility', 'TotalFertilityFormula', 'LengthofMenses',
       'MeanMensesLength', 'MensesScoreDayOne', 'MensesScoreDayTwo',
       'MensesScoreDayThree', 'MensesScoreDayFour', 'MensesScoreDayFive',
       'MensesScoreDaySix', 'MensesScoreDaySeven', 'MensesScoreDayEight',
       'MensesScoreDayNine', 'MensesScoreDayTen', 'MensesScoreDay11',
       'MensesScoreDay12', 'MensesScoreDay13', 'MensesScoreDay14',
       'MensesScoreDay15', 'TotalMensesScore', 'MeanBleedingIntensity',
       'NumberofDaysofIntercourse', 'IntercourseInFertileWindow',
       'UnusualBleeding', 'PhasesBleeding', 'IntercourseDuringUnusBleed',
       'Age', 'AgeM', 'Maristatus', 'MaristatusM', 'Yearsmarried', 'Wedding

In [6]:
combined_data['CorrectNextPeriodDate'] = combined_data['CorrectNextPeriodDate'].str.strip()

KeyError: 'CorrectNextPeriodDate'

In [ ]:
combined_dataset = pd.read_csv("Combined_dataset.csv")

# Select the relevant columns for the input features and target variable
X = combined_dataset[['ClientAge', 'ClientWeight', 'LengthOfCycle', 'ClientHeight', 'ClientBMI']]
y = combined_dataset['NextPeriodDate']

# Train a linear regression model
model = LinearRegression()
model.fit(X, y)

# Prepare the user's data for prediction
user_data = {
    "ClientID": [ClientId],
    "CycleNumber": [CycleNumber],
    "LengthOfCycle": [LengthOfCycle],
    "ClientWeight": [Weight],
    "ClientAge":[Age],
    "ClientBMI":[BMI],
}
user_data = pd.DataFrame(user_data)

# Predict the next period date for the user
next_period_date = model.predict(user_data)

print("Predicted Next Period Date:", next_period_date)

KeyError: 'NextPeriodDate'

In [10]:
# Split the data into initial training set and streaming data set
n_initial_samples = 100  # Number of initial training samples
initial_train_data = combined_dataset[:n_initial_samples]
streaming_data = combined_dataset[n_initial_samples:]


# Fit the initial model
degree = 2  # Degree of polynomial features
poly_features = PolynomialFeatures(degree=degree)
X_initial_train = poly_features.fit_transform(initial_train_data[['IndependentVariable']])
y_initial_train = initial_train_data['DependentVariable']

model = LinearRegression()
model.fit(X_initial_train, y_initial_train)

# Iterate over the streaming data and update the model
for i in range(len(streaming_data)):
    # Get the next sample from the streaming data set
    sample = streaming_data.iloc[i]

    # Update the model with the new sample
    x = np.array(sample['IndependentVariable']).reshape(1, -1)
    X_poly = poly_features.transform(x)
    y = np.array(sample['DependentVariable'])
    model.partial_fit(X_poly, [y])

# Evaluate the updated model on the streaming data set
X_stream = poly_features.transform(streaming_data[['IndependentVariable']])
y_stream = streaming_data['DependentVariable']
y_pred_stream = model.predict(X_stream)

# Calculate evaluation metrics
mse_stream = mean_squared_error(y_stream, y_pred_stream)
r2_stream = r2_score(y_stream, y_pred_stream)

KeyError: "None of [Index(['IndependentVariable'], dtype='object')] are in the [columns]"

In [ ]:
for i in range(len(streaming_data)):
    # Get the next sample from the streaming data set
    sample = streaming_data.iloc[i]

    # Update the model with the new sample
    x = np.array(sample['IndependentVariable']).reshape(1, -1)
    y = np.array(sample['DependentVariable'])
    model.partial_fit(x, [y])

In [ ]:
# Evaluate the updated model on the streaming data set
x_stream = streaming_data[['IndependentVariable']]
y_stream = streaming_data['DependentVariable']
y_pred_stream = model.predict(x_stream)

# Calculate evaluation metrics
mse_stream = mean_squared_error(y_stream, y_pred_stream)
r2_stream = r2_score(y_stream, y_pred_stream)